In [1]:
import ee, datetime
import pandas as pd
import numpy as np
import folium
import geehydro
import datetime as dt
import IPython.display


In [2]:

ee.Authenticate()
ee.Initialize()


Successfully saved authorization token.


Selecting the area of Interest

In [3]:
lower_Lake = folium.Map(location=[23.248905930238088, 77.4059219962347], zoom_start=10)


In [4]:
lower_Lake

Selecting the satellite and AOI

In [5]:
sentinel = ee.ImageCollection("COPERNICUS/S2_SR").\
               filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20)).\
               filterDate("2015-01-01","2023-01-01")
lower_Lake_AOI = ee.Geometry.Polygon([[[77.40540829355803,23.250265289120904],[77.40497914011564,23.250107568377107],[77.40454998667326,23.24987098691169],[77.40394917185392,23.249397822721765],[77.40296211893644,23.249634405026583],[77.40188923533049,23.250068138162035],[77.40124550516691,23.25014699858056],[77.40030136759368,23.25006813816203],[77.39922848398771,23.250738450233026],[77.39828434641447,23.250975030159537],[77.3976406162509,23.25046243978831],[77.39802685434904,23.249910417185077],[77.39879933054533,23.249358392296827],[77.39957180674162,23.249042948477634],[77.40017262156096,23.248688073289212],[77.40068760569181,23.248490919998734],[77.40128842051116,23.248057181733795],[77.40214672739592,23.247584011108767],[77.40261879618254,23.247307994135603],[77.40334835703459,23.247071407703988],[77.40407791788664,23.2469136831831],[77.4051508014926,23.246598233581782],[77.40480747873869,23.245888469251092],[77.40429249460783,23.245415290932417],[77.40356293375578,23.245060406091877],[77.40274754221525,23.244390065483675],[77.40214672739592,23.244035177915574],[77.40150299723234,23.24344369653732],[77.40154591257658,23.242852212536526],[77.40227547342865,23.243049374161522],[77.40283337290373,23.243443696537323],[77.40334835703459,23.24360142516129],[77.40407791788664,23.24387744980447],[77.40472164805021,23.244390065483675],[77.40523663218109,23.244666088495027],[77.40588036234465,23.24490267919301],[77.40635243113127,23.24569131182231],[77.40738239939299,23.246203920530995],[77.40798321421232,23.246203920530995],[77.40849819834318,23.24608562638846],[77.40858402903166,23.247426287194042],[77.40901318247404,23.248214904902426],[77.40944233591642,23.248964087406257],[77.41000023539152,23.25006813816203],[77.41072979624357,23.250608695872465],[77.41012898142424,23.250805846032197],[77.41060105021086,23.25139729476258],[77.41111603434172,23.25171273301285],[77.4121030872592,23.251633873520237],[77.41287556345549,23.25151558419388],[77.41381970102873,23.251160715585222],[77.41377678568449,23.25147615439511],[77.41364803965178,23.252028170516947],[77.41326180155363,23.252580184353608],[77.4126609867343,23.252974478552105],[77.41177199151964,23.253329342333874],[77.41155741479845,23.25376306345126],[77.41108534601183,23.25407849610504],[77.41065619256945,23.25415735415188],[77.40958330896349,23.254630501453526],[77.4090254094884,23.25447278587288],[77.40889666345568,23.254196783157816],[77.40859625604601,23.253684205171215],[77.40812418725939,23.253171625214122],[77.40773794916124,23.252737902172907],[77.40726588037462,23.252383036817132],[77.40675089624376,23.25194931121088],[77.40615008142443,23.251633873520237],[77.40567801263781,23.251279005226362],[77.40584967401476,23.250529835726965],[77.40540829355803,23.250265289120904]]])

sentinel_AOI = sentinel.filterBounds(lower_Lake_AOI)
               

Total Number of Images in Image Collection

In [6]:
print('Total number of images :', sentinel_AOI.size().getInfo())

Total number of images : 166


# *NDCI Calculation*

In [7]:
def calculate_NDCI(image):
  ndci = image.normalizedDifference(['B5', 'B4']).rename('NDCI')
  return image.addBands(ndci)
ndci = sentinel_AOI.map(calculate_NDCI)  

*Calculating mean of NDCI on every image present in the image collection*

In [8]:
def calculate_mean_NDCI(image):
  image = ee.Image(image)
  mean = image.reduceRegion(reducer = ee.Reducer.mean().setOutputs(['NDCI']),
                           geometry = lower_Lake_AOI,
                           scale = image.projection().nominalScale().getInfo(),
                           maxPixels = 100000,
                           bestEffort = True);
  return mean.get('NDCI').getInfo()

In [9]:
Images_ndci = ndci.select('NDCI').toList(ndci.size())
ndci_coll = []
for i in range(Images_ndci.length().getInfo()):
  image = ee.Image(Images_ndci.get(i-1))
  temp_ndci = calculate_mean_NDCI(image)
  ndci_coll.append(temp_ndci)

KeyboardInterrupt: 

*fetching Dates from the Image Collection*


In [10]:
dates = np.array(ndci.aggregate_array("system:time_start").getInfo())
day = [datetime.datetime.fromtimestamp(i/1000).strftime('%Y-%m-%d') for i in (dates)]

Creating the Dataframe

In [11]:
df = pd.DataFrame(ndci_coll, index = day, columns = ['Chlorophyll'])
df.index = pd.to_datetime(df.index, format="%Y/%m/%d")
df.sort_index(ascending = True, inplace = True)

In [12]:
df

,Chlorophyll
2018-12-18,0.040679
2018-12-23,0.068569
2018-12-28,0.192236
2019-01-02,0.133367
2019-01-07,0.078596
...,...
2022-12-02,0.032937
2022-12-07,0.037135
2022-12-17,0.041873
2022-12-22,0.055047


In [13]:
df.to_csv('Chlorophyll.csv')

## **Calculating NDTI**

In [14]:
def calculate_NDTI(image):
  ndti = image.normalizedDifference(['B4', 'B3']).rename('NDTI')
  return image.addBands(ndti)
ndti = sentinel_AOI.map(calculate_NDTI)

In [15]:
def calculate_mean_NDTI(image):
  image = ee.Image(image)
  mean = image.reduceRegion(reducer = ee.Reducer.mean().setOutputs(['NDTI']),
                           geometry = lower_Lake_AOI,
                           scale = image.projection().nominalScale().getInfo(),
                           maxPixels = 100000,
                           bestEffort = True);
  return mean.get('NDTI').getInfo()

In [16]:
Images_ndti = ndti.select('NDTI').toList(ndti.size())
ndti_coll = []
for i in range(Images_ndti.length().getInfo()):
  image = ee.Image(Images_ndti.get(i-1))
  temp_ndti = calculate_mean_NDTI(image)
  ndti_coll.append(temp_ndti)

In [17]:
dates = np.array(ndti.aggregate_array("system:time_start").getInfo())
day = [datetime.datetime.fromtimestamp(i/1000).strftime('%Y-%m-%d') for i in (dates)]

In [18]:
df1 = pd.DataFrame(ndti_coll, index = day, columns = ['Turbidity'])
df1.index = pd.to_datetime(df1.index, format="%Y/%m/%d")
df1.sort_index(ascending = True, inplace = True)

In [19]:
df1

,Turbidity
2018-12-18,-0.007191
2018-12-23,-0.111245
2018-12-28,-0.187315
2019-01-02,-0.150592
2019-01-07,-0.153835
...,...
2022-12-02,-0.013329
2022-12-07,-0.016475
2022-12-17,-0.006209
2022-12-22,-0.010468


In [20]:
df1.to_csv('Turbidity.csv')

## ** Calculating NDSI**

In [21]:
def calculate_NDSI(image):
  ndsi = image.normalizedDifference(['B11', 'B12']).rename('NDSI')
  return image.addBands(ndsi)
ndsi = sentinel_AOI.map(calculate_NDSI)

In [22]:
def calculate_mean_NDSI(image):
  image = ee.Image(image)
  mean = image.reduceRegion(reducer = ee.Reducer.mean().setOutputs(['NDSI']),
                           geometry = lower_Lake_AOI,
                           scale = image.projection().nominalScale().getInfo(),
                           maxPixels = 100000,
                           bestEffort = True);
  return mean.get('NDSI').getInfo()

In [23]:
Images_ndsi = ndsi.select('NDSI').toList(ndsi.size())
ndsi_coll = []
for i in range(Images_ndsi.length().getInfo()):
  image = ee.Image(Images_ndsi.get(i-1))
  temp_ndsi = calculate_mean_NDSI(image)
  ndsi_coll.append(temp_ndsi)

In [24]:
dates = np.array(ndsi.aggregate_array("system:time_start").getInfo())
day = [datetime.datetime.fromtimestamp(i/1000).strftime('%Y-%m-%d') for i in (dates)]

In [25]:
df2 = pd.DataFrame(ndsi_coll, index = day, columns = ['Salinty'])
df2.index = pd.to_datetime(df2.index, format="%Y/%m/%d")
df2.sort_index(ascending = True, inplace = True)

In [26]:
df2

,Salinty
2018-12-18,0.032208
2018-12-23,0.177992
2018-12-28,0.040495
2019-01-02,0.093497
2019-01-07,0.064393
...,...
2022-12-02,0.014207
2022-12-07,0.011204
2022-12-17,0.011836
2022-12-22,0.012525


In [27]:
df2.to_csv('Salinty.csv')

 *Dissloved Oxygen Calculation*

In [28]:
def calculate_DO(image):
  do = image.normalizedDifference(['B4', 'B5']).rename('DO')
  return image.addBands(do)
do = sentinel_AOI.map(calculate_DO)

In [29]:
def calcualte_mean_DO(image):
  image = ee.Image(image)
  mean = image.reduceRegion(reducer = ee.Reducer.mean().setOutputs(['DO']),
                           geometry = lower_Lake_AOI,
                           scale = image.projection().nominalScale().getInfo(),
                           maxPixels = 100000,
                           bestEffort = True);
  return mean.get('DO').getInfo()

In [30]:
Images_do = do.select('DO').toList(do.size())
do_coll = []
for i in range(Images_do.length().getInfo()):
  image = ee.Image(Images_do.get(i-1))
  temp_do = calcualte_mean_DO(image)
  do_coll.append(temp_do)

In [31]:
dates = np.array(do.aggregate_array("system:time_start").getInfo())
day = [datetime.datetime.fromtimestamp(i/1000).strftime('%Y-%m-%d') for i in (dates)]

In [32]:
df4 = pd.DataFrame(do_coll, index = day, columns = ['Dissolved Oxygen'])
df4.index = pd.to_datetime(df4.index, format="%Y/%m/%d")
df4.sort_index(ascending = True, inplace = True)

In [33]:
df4

,Dissolved Oxygen
2018-12-18,-0.041608
2018-12-23,-0.071488
2018-12-28,-0.197621
2019-01-02,-0.139806
2019-01-07,-0.082936
...,...
2022-12-02,-0.034098
2022-12-07,-0.038263
2022-12-17,-0.043224
2022-12-22,-0.056287


In [34]:
df4.to_csv('Dissolved Oxygen.csv')

# **pH**

In [35]:
def calculate_pH(image):
  ph = ee.Image(8.339).subtract(ee.Image(0.827).multiply(image.select('B1').divide(image.select('B8')))).rename('PH')
  return image.addBands(ph)
pH = sentinel_AOI.map(calculate_pH)

In [36]:
def calculate_mean_pH(image):
  image = ee.Image(image)
  mean = image.reduceRegion(reducer = ee.Reducer.mean().setOutputs(['PH']),
                           geometry = lower_Lake_AOI,
                           scale = image.projection().nominalScale().getInfo(),
                           maxPixels = 100000,
                           bestEffort = True);
  return mean.get('PH').getInfo()

In [37]:
Images_ph = pH.select('PH').toList(pH.size())
ph_coll= []
for i in range(Images_ph.length().getInfo()):
  image = ee.Image(Images_ph.get(i-1))
  temp_ph = calculate_mean_pH(image)
  ph_coll.append(temp_ph)

In [38]:
dates = np.array(pH.aggregate_array("system:time_start").getInfo())
day = [datetime.datetime.fromtimestamp(i/1000).strftime('%Y-%m-%d') for i in (dates)]

In [39]:
df6 = pd.DataFrame(ph_coll, index = day, columns = ['pH'])
df6.index = pd.to_datetime(df6.index, format="%Y/%m/%d")
df6.sort_index(ascending = True, inplace = True)

In [40]:
df6

,pH
2018-12-18,7.567741
2018-12-23,7.174907
2018-12-28,7.414977
2019-01-02,7.365687
2019-01-07,7.508085
...,...
2022-12-02,7.506434
2022-12-07,7.559180
2022-12-17,7.614748
2022-12-22,7.544612


In [41]:
df6.to_csv('pH.csv')

# **Temperature**


Selecting the Satellite



In [42]:
landsat = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2").\
              filterDate("2016-01-01","2022-01-01")
sarapangi_AOI = ee.Geometry.Polygon([[[77.42117255516425,23.20909792227606],[77.42057174034491,23.20799353262676],[77.42040007896796,23.207520219982936],[77.4200996715583,23.2069285768207],[77.41949885673895,23.207125791499145],[77.41902678795233,23.207007462726995],[77.41889804191962,23.206258044736966],[77.41868346519843,23.20550862254438],[77.41868346519843,23.204995857570196],[77.41851180382147,23.204680308915954],[77.41816848106757,23.203970321719954],[77.41885512657538,23.20314199855786],[77.41928428001776,23.202787001345527],[77.4204429943122,23.201840337503217],[77.42082923241034,23.201406447668166],[77.42125838585272,23.20089366695645],[77.42173045463934,23.20101200114144],[77.4224600154914,23.20140644766817],[77.42327540703192,23.20152478139923],[77.42361872978583,23.20187978196384],[77.42366164513007,23.202313670262438],[77.42413371391669,23.202589780264603],[77.42464869804755,23.203209392314182],[77.42507785148993,23.203761608047508],[77.42524951286688,23.20451104003633],[77.42451995201483,23.20569434514694],[77.42387622185126,23.206207107441198],[77.42348998375311,23.20707485453413],[77.42306083031075,23.208179251775938],[77.42284625358954,23.208692004535596],[77.42207377739327,23.20853423466525],[77.42143004722968,23.20837646460864],[77.42117255516425,23.20909792227606]]])

landsat_AOI = landsat.filterBounds(sarapangi_AOI)

In [43]:
print('Total number of images :', landsat_AOI.size().getInfo())

Total number of images : 227


In [44]:
def calculate_temp(image):
  temp = image.select('ST_B.*').multiply(0.00341802).add(149.0).subtract(273.15).rename('temperature')
  return image.addBands(temp)
temp = landsat_AOI.map(calculate_temp)


In [45]:
def calculate_mean_temp(image):
  image = ee.Image(image)
  mean = image.reduceRegion(reducer = ee.Reducer.mean().setOutputs(['temperature']),
                           geometry = lower_Lake_AOI,
                           scale = image.projection().nominalScale().getInfo(),
                           maxPixels = 100000,
                           bestEffort = True);
  return mean.get('temperature').getInfo()

In [46]:
Images_temp = temp.select('temperature').toList(temp.size())
temp_coll = []
for i in range(Images_temp.length().getInfo()):
  image = ee.Image(Images_temp.get(i-1))
  tempe_temp = calculate_mean_temp(image)
  temp_coll.append(tempe_temp)

In [47]:
dates = np.array(temp.aggregate_array("system:time_start").getInfo())
day = [datetime.datetime.fromtimestamp(i/1000).strftime('%Y-%m-%d') for i in (dates)]

In [48]:
df5 = pd.DataFrame(temp_coll, index = day, columns = ['Temperature'])
df5.index = pd.to_datetime(df5.index, format="%Y/%m/%d")
df5.sort_index(ascending = True, inplace = True)

In [49]:
df5

,Temperature
2016-01-07,18.444055
2016-01-14,21.731807
2016-01-23,22.069304
2016-01-30,21.538733
2016-02-08,20.412248
...,...
2021-11-27,24.033039
2021-12-06,26.843447
2021-12-13,25.127994
2021-12-22,24.733931


In [50]:
df5.to_csv('Temperature.csv')